# Воркшоп #7: Altair, робота з географією


### ПОЧАТОК ПІДГОТОВЧОЇ ЧАСТИНИ



Цей воркшоп виконує...

### КІНЕЦЬ ПІДГОТОВЧОЇ ЧАСТИНИ
-----------------------------
### ПОЧАТОК СПІЛЬНОЇ ЧАСТИНИ

Спершу імпортуємо усе необхідне:

In [ ]:
import altair as alt
import pandas as pd
from vega_datasets import data

Почнемо! 

Перш за все, давайте намалюємо політичну карту світу. У `vega_datasets` є підходящий GeoJSON, у датасеті `world110m`. Сама "фіча" називається `countries`, тож і зробити `topo_feature` досить просто:

Щоб трохи полегшити подальшу роботу, ознайомтесь з файлом, який задає карту світу. За потреби скористайтесь [json prettifier](https://jsonformatter.curiousconcept.com/#). 

Тепер використайте стандартний підхід Altair (через `Chart`), щоб візуалізувати карту світу. 

За бажання, зробіть карту трошечки більшою, задавши іншу ширину :) 

In [ ]:
world_map = alt.topo_feature(data.world_110m.url, 'countries')

print(data.world_110m.url)

https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/world-110m.json


In [ ]:
better_world = world_map

Карта прекрасна, але щось у ній зайве. Використовуючи ось [цю](https://en.wikipedia.org/wiki/ISO_3166-1_numeric) табличку, а також `transform_filter` та `alt.datum`, виведіть кращий світ, світ без рф.

In [ ]:
density_df = pd.read_csv('https://raw.githubusercontent.com/tillnagel/unfolding/refs/heads/master/data/data/countries-population-density.csv', delimiter=';')
country_codes = country_codes[['alpha-3', 'name', 'country-code', 'region']]

merged_df = density_df.merge(country_codes, left_on='Country Code', right_on='alpha-3')
merged_df

,Country Name,Country Code,2010,alpha-3,name,country-code,region
0,Afghanistan,AFG,52.719133,AFG,Afghanistan,4,Asia
1,Albania,ALB,116.970803,ALB,Albania,8,Europe
2,Algeria,DZA,14.891634,DZA,Algeria,12,Africa
3,American Samoa,ASM,342.100000,ASM,American Samoa,16,Oceania
4,Andorra,AND,180.561702,AND,Andorra,20,Europe
...,...,...,...,...,...,...,...
209,Virgin Islands (U.S.),VIR,314.285714,VIR,Virgin Islands (U.S.),850,Americas
210,West Bank and Gaza,PSE,689.700997,PSE,"Palestine, State of",275,Asia
211,Yemen. Rep.,YEM,45.557513,YEM,Yemen,887,Asia
212,Zambia,ZMB,17.389257,ZMB,Zambia,894,Africa


Тепер спробуємо використати трохи публічних даних і зобразити їх на мапі візуально.

Використаємо два датасети:
- Дані про населення - https://github.com/tillnagel/unfolding/blob/master/data/data/countries-population-density.csv (зверніть увагу, що вам треба отримати `raw` посилання і зчитати його за допомогою pandas). 
- Дані про коди країн (трьохбуквенні коди і ISO-3166-1 коди) - https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/blob/master/all/all.csv

Отож: спробуйте у кольорі закодувати кількість населення. Вам у цьому знадобиться функція `transform_lookup` (приклад використання можна знайти [тут](https://altair-viz.github.io/gallery/choropleth.html))


**На випадок, якщо у вас помилка верифікації SSL-зʼєднання при спробі скачати csv файл через pandas, наступний код має допомогти:**

```
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
```



In [ ]:
# Щоб було легше, код розділимо на дві половини.
# У цій комірці зчитайте датасети, відфільтруйте та зʼєднайте з використанням pd.merge
density_df = pd.read_csv('https://raw.githubusercontent.com/tillnagel/unfolding/refs/heads/master/data/data/countries-population-density.csv', delimiter=';')
density_df.rename(columns={'Country Name':'country', '2010':'density'}, inplace=True)
density_df 

country_codes_filtered = country_codes[['alpha-3','name', 'country-code', 'region']]

merged_df = density_df.merge(country_codes_filtered, left_on='Country Code', right_on='alpha-3')
merged_df

,country,Country Code,density,alpha-3,name,country-code,region
0,Afghanistan,AFG,52.719133,AFG,Afghanistan,4,Asia
1,Albania,ALB,116.970803,ALB,Albania,8,Europe
2,Algeria,DZA,14.891634,DZA,Algeria,12,Africa
3,American Samoa,ASM,342.100000,ASM,American Samoa,16,Oceania
4,Andorra,AND,180.561702,AND,Andorra,20,Europe
...,...,...,...,...,...,...,...
209,Virgin Islands (U.S.),VIR,314.285714,VIR,Virgin Islands (U.S.),850,Americas
210,West Bank and Gaza,PSE,689.700997,PSE,"Palestine, State of",275,Asia
211,Yemen. Rep.,YEM,45.557513,YEM,Yemen,887,Asia
212,Zambia,ZMB,17.389257,ZMB,Zambia,894,Africa


In [ ]:
# У цій комірці займіться, власне, візуалізацією

density_map = alt.Chart(better_world).mark_geoshape().encode(
    color=alt.Color("density:Q", scale=alt.Scale(scheme="teals"))
).transform_lookup(
    lookup="id",
    from_=alt.LookupData(merged_df, key="country-code", fields=["density", "name", "country-code", "region"])
)

density_map

alt.Chart(...)

Чудово! Давайте додатково додамо підказку при наведенні на країну, яка б містила назву країни та щільність населення.

Крім того, додамо назву, а шкалу підпишемо і використаємо схему зафарбування `teals`. До речі, список схем доступний [ось тут](https://vega.github.io/vega/docs/schemes/)

Тепер додамо трохи інтерактиву.

Спробуйте створити повзунок, який дасть змогу візуалізувати виключно ті країни, у яких щільність менша за конкретно вибрану.

Вам у цьому допоможе `binding_range` та `param`.

In [ ]:
min_slider = alt.binding_range(min=0, max=1200, step=1, name="Min density")
max_slider = alt.binding_range(min=0, max=1200, step=1, name="Min density")

max_param = alt.param(name="max", value=1200, bind=max_slider)
min_param = alt.param(name="max", value=1200, bind=min_slider)

density_map = density_map.transform_filter(
    (alt.datum.density > min_param) & (alt.datum.density < max_param)
).add_params(min_param, max_param)

density_map

alt.Chart(...)

Чудово! А тепер давайте ще додамо можливість вибрати конкретний континент :)

In [ ]:
continents = alt.binding_select(name="continets", option=["Asia","Europa","Africa","Oceania","Americas"])
continents_param = alt.param(name="continent", value="Americas", bind=continents)

density_map = density_map.transform_filter(
    (alt.datum.density > min_param) & (alt.datum.density < max_param)
).add_params(min_param, max_param)
density_map

SchemaValidationError: `BindRadioSelect` has no parameter named 'option'

Existing parameter names are:
input     debounce   labels   
options   element    name     

See the help for `BindRadioSelect` to read the full description of these parameters

In [ ]:
merged_df

Прекрасно!

------------

Давайте повернемося до "простої" карти із зображенням світу. 

Давайте спробуємо зобразити на карті інтернет-кабелі. Ось потрібний датасет - https://github.com/lifewinning/submarine-cable-taps/blob/master/data/submarine_cables.geojson

Отже, завдання: зобразіть карту світу, на ній інтернет-кабелі. Використайте адекватну кольорову гаму, 

Звісно, на карту можна наносити і прості обʼєкти - точки, площі.

Нам знадобиться карта з [цього](https://github.com/org-scn-design-studio-community/sdkcommunitymaps) репозиторію, а також дані з файлу [full_dataset.csv](https://raw.githubusercontent.com/kse-ua/KSE-Loc-Data-Hub/refs/heads/main/data/derived/full_dataset.csv) (джерело - [KSE-Loc-Data-Hub](https://github.com/kse-ua/KSE-Loc-Data-Hub/tree/main)).

Спробуйте зобразити центри громад з найбільшою площею - скажімо, топ-50 громад.

In [ ]:
ukraine_map = alt.top0_feature(https://raw.github.com/org-scn-design-studio-community/sdkcommunitymaps/blob/master/geojson/Europe/Ukraine-regions.json)

In [ ]:
hromads = pd.read_csv("full_dataset")
list(centres,columns)

In [ ]:
hromads_50 = hromads.sort_values("square", ascending=False).iloc[:50]
centres_map = alt.Chart(hromads_50).mark_point(color="orange").encode(
    longitude="lon center"
    latitude="lat_centre"
    toolip=["hromada_name"]
)
hromads_50

А як щодо візуалізації доходів громад від доходів фізичних осіб у 2022 році? Спробуйте передати взаємозвʼязок між доходами та загальною кількістю населення у 2022 році

In [ ]:
map_1 = alt.Chart(hromads_50).mark_point(color="orange").encode(
    longitude="lon_centre"
    latitude="lat_center"
    color="income_total_2022"
    size="total_popultation_2022"
    toolip=["hromada_name"]
)
ukraine_map + map_1

### КІНЕЦЬ СПІЛЬНОЇ ЧАСТИНИ

-----------------------------
### ПОЧАТОК ДОМАШНЬОЇ ЧАСТИНИ

У якості домашньої частини необхідно візуалізувати будь-які дані з використанням мапи. Тобто взагалі будь-які, які спадуть вам на думку і датасети до яких ви зможете знайти.